# Fit a global model on the data

- For any given year of data, the outcome of a game can be described as a bayesian model of the number of points

In [1]:
# --- notebook cell 1 ---
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, List, Tuple, Iterable, TypedDict, Optional

import numpy as np
import pandas as pd
import jax
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS

# For better numerical stability in Poisson log-rates
jax.config.update("jax_enable_x64", True)
numpyro.set_host_device_count(1)


## Utils

In [7]:
class SeasonDF(TypedDict):
    gameID: int
    date: str
    home_team: str
    away_team: str
    home_score: int
    away_score: int
    finalMessage: str
    start_time: str
    url: str
    conference_home: str
    conference_away: str

@dataclass(frozen=True)
class EncodedSeason:
    home_idx: jnp.ndarray     # shape (n_games,), int32
    away_idx: jnp.ndarray     # shape (n_games,), int32
    y_home: jnp.ndarray       # shape (n_games,), int32
    y_away: jnp.ndarray       # shape (n_games,), int32
    n_teams: int
    id_to_team: List[str]
    team_to_id: Dict[str, int]


In [8]:
def _normalize_team_name(name: str) -> str:
    # light normalization to avoid accidental dupes
    return " ".join(name.strip().split())

def build_team_indexer(df: pd.DataFrame) -> Tuple[Dict[str, int], List[str]]:
    assert {"home_team", "away_team"}.issubset(df.columns), "Missing required columns."
    teams: List[str] = sorted({
        _normalize_team_name(t) for t in
        df["home_team"].astype(str).tolist() + df["away_team"].astype(str).tolist()
    })
    team_to_id = {t: i for i, t in enumerate(teams)}
    id_to_team = teams
    return team_to_id, id_to_team

def encode_season(df: pd.DataFrame, team_to_id: Dict[str, int]) -> EncodedSeason:
    # ensure required columns present and typed
    req = {"home_team","away_team","home_score","away_score"}
    missing = req - set(df.columns)
    if missing:
        raise ValueError(f"DataFrame missing columns: {missing}")

    home_idx = df["home_team"].astype(str).map(lambda x: team_to_id[_normalize_team_name(x)]).astype(np.int32).to_numpy()
    away_idx = df["away_team"].astype(str).map(lambda x: team_to_id[_normalize_team_name(x)]).astype(np.int32).to_numpy()

    y_home = df["home_score"].astype(np.int32).to_numpy()
    y_away = df["away_score"].astype(np.int32).to_numpy()

    n_teams = len(team_to_id)
    id_to_team = [None] * n_teams  # type: ignore
    for t, i in team_to_id.items():
        id_to_team[i] = t  # fill by index

    return EncodedSeason(
        home_idx=jnp.array(home_idx),
        away_idx=jnp.array(away_idx),
        y_home=jnp.array(y_home),
        y_away=jnp.array(y_away),
        n_teams=n_teams,
        id_to_team=id_to_team,  # type: ignore
        team_to_id=team_to_id,
    )


In [9]:
def _cov_cholesky(scale_off: jnp.ndarray, scale_def: jnp.ndarray, rho: jnp.ndarray) -> jnp.ndarray:
    S = jnp.array([
        [scale_off**2,           rho*scale_off*scale_def],
        [rho*scale_off*scale_def, scale_def**2          ],
    ])
    return jnp.linalg.cholesky(S)

def model_hier_offdef_home(
    home_idx: jnp.ndarray,
    away_idx: jnp.ndarray,
    y_home: jnp.ndarray,
    y_away: jnp.ndarray,
    n_teams: int,
):
    # global intercept absorbs level
    alpha = numpyro.sample("alpha", dist.Normal(0., 2.))

    # hyper-means
    mu_off = numpyro.sample("mu_off", dist.Normal(0., 1.))
    mu_def = numpyro.sample("mu_def", dist.Normal(0., 1.))
    mu = jnp.array([mu_off, mu_def])

    # hyper-scales & correlation
    scale_off = numpyro.sample("scale_off", dist.HalfNormal(1.))
    scale_def = numpyro.sample("scale_def", dist.HalfNormal(1.))
    rho       = numpyro.sample("rho", dist.Uniform(-0.95, 0.95))
    L = _cov_cholesky(scale_off, scale_def, rho)

    # home effect hyperparameters (global)
    h_mu = numpyro.sample("h_mu", dist.Normal(0., 0.5))
    h_sd = numpyro.sample("h_sd", dist.HalfNormal(0.5))

    with numpyro.plate("team", n_teams):
        theta = numpyro.sample("theta", dist.MultivariateNormal(mu, scale_tril=L))  # (n_teams,2)
        h     = numpyro.sample("h", dist.Normal(h_mu, h_sd))                        # (n_teams,)

    off = theta[:, 0]
    deff = theta[:, 1]

    # log-rates (Poisson means = exp(eta))
    eta_home = alpha + off[home_idx] - deff[away_idx] + h[home_idx]
    eta_away = alpha + off[away_idx] - deff[home_idx] - h[home_idx]

    numpyro.sample("y_home", dist.Poisson(jnp.exp(eta_home)), obs=y_home)
    numpyro.sample("y_away", dist.Poisson(jnp.exp(eta_away)), obs=y_away)


## Global prior

In [10]:
def fit_global_hypers(encoded: EncodedSeason, seed: int = 0, num_chains: int = 4,
                      num_warmup: int = 1000, num_samples: int = 10_000):
    kernel = NUTS(model_hier_offdef_home)
    mcmc = MCMC(kernel, num_warmup=num_warmup, num_chains=num_chains, num_samples=num_samples, progress_bar=True)
    rng = jax.random.PRNGKey(seed)
    mcmc.run(
        rng,
        home_idx=encoded.home_idx,
        away_idx=encoded.away_idx,
        y_home=encoded.y_home,
        y_away=encoded.y_away,
        n_teams=encoded.n_teams,
    )
    post = mcmc.get_samples()
    # EB plug-in (posterior means)
    eb = {k: post[k].mean(axis=0) for k in
          ["alpha","mu_off","mu_def","scale_off","scale_def","rho","h_mu","h_sd"]}
    return eb, mcmc


In [15]:
df = pd.read_csv("/Users/arhamhabib/Projects/PostPick/data/ncaab_2021_men_d1.csv")
df.dropna(inplace=True, subset=['home_team', 'away_team', 'home_score', 'away_score'])
df

,gameID,date,home_team,away_team,home_score,away_score,finalMessage,start_time,url,conference_home,conference_away
0,1976847.0,2021-11-01,Florida,Embry-Riddle (FL),80.0,57.0,FINAL,12:00AM ET,5896325,SEC,Sunshine State
1,1976852.0,2021-11-01,Alcorn,Xavier (LA),67.0,45.0,FINAL,01:00AM ET,5896329,SWAC,NON-NCAA ORG
2,1976853.0,2021-11-01,Texas,Texas Lutheran,96.0,33.0,FINAL,01:00AM ET,5896330,Big 12,SCAC
3,1976854.0,2021-11-01,Oklahoma,Rogers St.,106.0,57.0,FINAL,01:00AM ET,5896331,Big 12,Mid-America Intercollegiate
4,1994143.0,2021-11-01,UAB,AUM,101.0,70.0,FINAL,01:00AM ET,5907144,C-USA,Gulf South
...,...,...,...,...,...,...,...,...,...,...,...
6114,2070074.0,2022-03-31,Texas A&M,Xavier,72.0,73.0,FINAL,07:00PM ET,5996538,SEC,Big East
6115,2076366.0,2022-04-01,Coastal Carolina,Fresno St.,74.0,85.0,FINAL,06:00PM ET,5996410,Sun Belt,Mountain West
6116,2070510.0,2022-04-02,Kansas,Villanova,81.0,65.0,FINAL,06:09PM ET,5958444,Big 12,Big East
6117,2070511.0,2022-04-02,Duke,North Carolina,77.0,81.0,FINAL,08:51PM ET,5958443,ACC,ACC


In [ ]:
# 2) Build global index, encode, and fit EB hypers on the FULL SEASON (replace df with your full season df)
team_to_id, id_to_team = build_team_indexer(df)
encoded = encode_season(df, team_to_id)
eb_hypers, global_mcmc = fit_global_hypers(encoded, seed=0, num_warmup=500, num_samples=1000)

/var/folders/fc/f6vjw8411pqfkmvl5w4gbj800000gn/T/ipykernel_29043/1413384060.py:4: UserWarning: There are not enough devices to run parallel chains: expected 4 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(4)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc = MCMC(kernel, num_warmup=num_warmup, num_chains=num_chains, num_samples=num_samples, progress_bar=True)
sample: 100%|██████████| 1500/1500 [00:27<00:00, 53.75it/s, 127 steps of size 3.64e-02. acc. prob=0.69] 


In [18]:
eb_hypers

{'alpha': Array(4.18792955, dtype=float64),
 'mu_off': Array(0.02343575, dtype=float64),
 'mu_def': Array(-0.04779102, dtype=float64),
 'scale_off': Array(0.14475448, dtype=float64),
 'scale_def': Array(0.15289272, dtype=float64),
 'rho': Array(0.74093184, dtype=float64),
 'h_mu': Array(0.01947409, dtype=float64),
 'h_sd': Array(0.00441662, dtype=float64)}

## Fit team posterior

In [19]:
def team_model_given_hypers(
    home_idx: jnp.ndarray,
    away_idx: jnp.ndarray,
    y_home: jnp.ndarray,
    y_away: jnp.ndarray,
    n_teams: int,
    hypers: Dict[str, jnp.ndarray],
):
    alpha = numpyro.deterministic("alpha", hypers["alpha"])
    mu = jnp.array([hypers["mu_off"], hypers["mu_def"]])
    L = _cov_cholesky(hypers["scale_off"], hypers["scale_def"], hypers["rho"])

    with numpyro.plate("team", n_teams):
        theta = numpyro.sample("theta", dist.MultivariateNormal(mu, scale_tril=L))
        h     = numpyro.sample("h", dist.Normal(hypers["h_mu"], hypers["h_sd"]))

    off, deff = theta[:, 0], theta[:, 1]
    eta_home = alpha + off[home_idx] - deff[away_idx] + h[home_idx]
    eta_away = alpha + off[away_idx] - deff[home_idx] - h[home_idx]

    numpyro.sample("y_home", dist.Poisson(jnp.exp(eta_home)), obs=y_home)
    numpyro.sample("y_away", dist.Poisson(jnp.exp(eta_away)), obs=y_away)

@dataclass(frozen=True)
class TeamPosterior:
    team_name: str
    off: jnp.ndarray   # samples
    deff: jnp.ndarray  # samples
    h: jnp.ndarray     # samples

def fit_team_posterior_from_df(
    team_df: pd.DataFrame,
    team_name: str,
    seed: int,
    eb_hypers: Dict[str, jnp.ndarray],
    num_warmup: int = 800,
    num_samples: int = 1200,
) -> TeamPosterior:
    # Re-index on the subset (team + opponents in that subset)
    tmap, _ = build_team_indexer(team_df)
    encoded = encode_season(team_df, tmap)
    if _normalize_team_name(team_name) not in tmap:
        raise ValueError(f"Team '{team_name}' not present in the provided dataframe.")

    mcmc = MCMC(NUTS(team_model_given_hypers), num_warmup=num_warmup, num_samples=num_samples, progress_bar=True)
    mcmc.run(
        jax.random.PRNGKey(seed),
        home_idx=encoded.home_idx,
        away_idx=encoded.away_idx,
        y_home=encoded.y_home,
        y_away=encoded.y_away,
        n_teams=encoded.n_teams,
        hypers=eb_hypers,
    )
    samp = mcmc.get_samples()
    theta = samp["theta"]            # shape: (S, n_teams, 2)
    h     = samp["h"]                # shape: (S, n_teams)

    tid = tmap[_normalize_team_name(team_name)]
    off_s = theta[:, tid, 0]
    def_s = theta[:, tid, 1]
    h_s   = h[:, tid]
    return TeamPosterior(team_name=team_name, off=off_s, deff=def_s, h=h_s)


In [21]:
# 3) Filter a single team's games and fit that team's posterior given EB hypers
target_team = "Duke"
team_games = df[(df["home_team"] == target_team) | (df["away_team"] == target_team)].copy()
team_post = fit_team_posterior_from_df(team_games, team_name=target_team, seed=1, eb_hypers=eb_hypers)

# team_post.off / team_post.deff / team_post.h are posterior samples (arrays)
float(team_post.off.mean()), float(team_post.deff.mean()), float(team_post.h.mean())

sample: 100%|██████████| 2000/2000 [00:01<00:00, 1981.34it/s, 15 steps of size 3.32e-01. acc. prob=0.89]


(0.1334990592798847, 0.008312732403002624, 0.018383336129210136)

In [22]:
team_post

TeamPosterior(team_name='Duke', off=Array([0.13211471, 0.13065726, 0.13292061, ..., 0.1287622 , 0.09235715,
       0.12752354], dtype=float64), deff=Array([ 0.01059702, -0.01592069, -0.00824106, ..., -0.00153763,
       -0.00829476,  0.01669756], dtype=float64), h=Array([0.01976944, 0.02083495, 0.01977011, ..., 0.01576341, 0.02156214,
       0.02589975], dtype=float64))